# Automating Excel with Python using xlwings


This comprehensive guide covers using Python and xlwings for Excel automation, 
from basics to advanced topics. We'll explore connecting to Excel workbooks, 
manipulating ranges, integrating pandas for data analysis, and creating dynamic examples 
like an animated bar chart. Python's flexibility and rich ecosystem make it a superior alternative to VBA.


## 1. Installing xlwings and Basic Setup

<p style="background:black">
<code style="background:black;color:white">C:\Users\YOUR_USERNAME> pip install xlwings
</code>
</p>

In [1]:
import xlwings as xw  # pip install xlwings
import pandas as pd   # pip install pandas

# Check the version of xlwings
xw.__version__

'0.33.4'


### Explanation
- **Installation:** Install xlwings using `pip install xlwings`.
- **Version Check:** Ensures xlwings is properly installed and up to date.


## 2. Connecting to an Excel Workbook

### Three different scenarios [New Book, Unsaved Book, Saved Book]

|  | xw.Book |
| :- | :- |
| New Book | xw.Book() |
| Unsaved book | xw.Book('Book1') |
| Book by (full)name | xw.Book(r'C:/path/to/file.xlsx')|


👉 **`xlwings`** Documentation: https://docs.xlwings.org/en/stable/connect_to_workbook.html

In [2]:
# Connect to Excel (comment/uncomment as needed)
wb = xw.Book()  # Create a new workbook
# wb = xw.Book('example.xlsx')  # Open an existing workbook

# Get the active sheet
sheet = wb.sheets.active

# Display the name of the active sheet
sheet.name

'Sheet1'


### Explanation
- **New Workbook:** Creates a blank Excel workbook.
- **Existing Workbook:** Opens a specified Excel file.
- **Active Sheet:** The current sheet where actions are performed.


## 3. Writing and Reading Data

In [3]:
# Write to a cell (comment/uncomment to test)
sheet.range("A1").value = "Hello, xlwings!"

# Read from a cell
value = sheet.range("A1").value
value

'Hello, xlwings!'


### Explanation
- **Write to Excel:** Assign values to cells or ranges.
- **Read from Excel:** Retrieve data from Excel for processing in Python.


## 4. Working with Ranges and Formulas

In [4]:
# Writing multiple values to a range
sheet.range("A2").value = [["Name", "Age"], ["Alice", 25], ["Bob", 30]]

# Reading multiple values from a range
data = sheet.range("A3:B4").value
data

# Writing a formula
sheet.range("C2").value = "=SUM(B3:B4)"


### Explanation
- **Ranges:** Supports writing and reading lists or tables.
- **Formulas:** xlwings allows dynamic formula creation for Excel calculations.


## 5. Integrating pandas for Data Analysis

In [6]:
# Create a pandas DataFrame
df = pd.DataFrame({
    "Product": ["A", "B", "C"],
    "Sales": [100, 150, 200]
})

# Write DataFrame to Excel (comment/uncomment to test)
sheet.range("A5").value = df

# Read data back into a DataFrame (comment/uncomment to test)
df_from_excel = sheet.range("B5").options(pd.DataFrame, expand="table", index=False).value
df_from_excel

,Product,Sales
0,A,50.0
1,B,10.0
2,C,30.0



### Explanation
- **DataFrame to Excel:** Directly write pandas DataFrames to Excel for visualization.
- **Excel to DataFrame:** Retrieve structured data for analysis in Python.


## 6. Closing the Workbook

In [7]:
# Close the workbook without saving
# wb.close()

# Or save the workbook
wb.save("automated_report.xlsx")
wb.close()

## 7. Important Remark: Use the context manager with xlwings

### Why Use a Context Manager with xlwings?

When automating Excel tasks with `xlwings`, it's essential to manage Excel application instances properly. Without a context manager, an error in your script could leave behind "zombie" Excel instances running in the background, consuming memory and resources.

By using a context manager (`with xw.App() as app:`), you ensure:

1. **Automatic Cleanup**: The Excel application is closed automatically when the block ends, even if an error occurs.
2. **Error Handling**: Prevents leftover processes, which can slow down your system and cause conflicts.
3. **Simpler Code**: No need to manually call `app.quit()`—the context manager handles it for you.

In [8]:
import xlwings as xw

# Open an Excel app as a context manager
with xw.App(visible=False) as app:  # Ensure the app runs in the background
    # Add a new workbook
    wb = app.books.add()

    # Get the active sheet
    sheet = wb.sheets[0]

    # Write data to the sheet
    sheet.range("A1").value = "Hello, xlwings!"

    # Read the written data
    print(sheet.range("A1").value)

    # Save the workbook
    wb.save("example_context_manager.xlsx")

    # Close the workbook
    wb.close()

# At this point, the app is automatically closed

Hello, xlwings!



### Explanation
- **Save:** Save changes to a file.
- **Close:** Close the workbook, optionally discarding changes.


## 8. Practical Example

In [9]:
# Install package into the instance of Python that launched the notebook
!pip install QuantStats  --quiet

DEPRECATION: Loading egg at c:\users\codin\appdata\local\programs\python\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [10]:
# Imports
from pathlib import Path
import quantstats as qs

In [11]:
# Specify excel file location (works on Mac & Windows)
filename = Path("stock_report/stock_report.xlsx")

# Open Workbook
wb = xw.Book(filename)

# Select Sheet
sht = wb.sheets['Report']

In [12]:
def generate_stock_report_as_pdf():
    
    # --- EXCEL DATA ---
    # Get the ticker symbol from Worksheet
    ticker = sht.range('TICKER_SYMBOL').value  
    
    ####################################################################
    
    # --- DO SOMETHING WITH THE EXCEL DATA IN PYTHON ---    
    # Fetch the daily returns for a stock 
    stock = qs.utils.download_returns(ticker)

    # Image location
    snapshot_path = Path('stock_report/snapshot.png')
    heatmap_path = Path('stock_report/return_heatmap.png')

    # Title of Snapshot Report
    title = f"{ticker} Performance"

    # Generate snapshot report of any given ticker symbol
    stock_snapshot = qs.plots.snapshot(stock,  
                                      title=title, 
                                      savefig=snapshot_path,
                                      show=False)

    # Generate montly return heatmap
    stock_heatmap = qs.plots.monthly_heatmap(stock, 
                                             savefig=heatmap_path,
                                             show=False)
    
    ######################################################################    
    
    # --- RETURN RESULT BACK TO EXCEL ---
    # Add Picutures to Excel
    sht.pictures.add(snapshot_path.absolute(), 
                     name='snapshot', 
                     update=True)
    sht.pictures.add(heatmap_path.absolute(), 
                     name='heatmap', 
                     update=True)

    # Save PDF Report
    wb.to_pdf('stock_report/stockreport')
    
    return None

In [13]:
# Generate Stock Report and save as PDF
generate_stock_report_as_pdf()

## Want to learn more?

<div class="alert alert-block alert-success">
If you’re ready to dive deeper into Excel automation and save hours on repetitive tasks, check out my <b>Excel Automation Course</b>.<br> It’s designed to help you work smarter and get more done.<br>
👉 <a href="https://pythonandvba.com/go/courses">Sign up here</a>!
</div>
